In [ ]:
import pandas as pd
df = pd.read_csv("fabric_capacity_units.csv")
df.head()

,Workspace,Item kind,Item name,CU (s),Duration (s),Users,Successful count,Failed count,Billing type
0,MDP-FABRIC-PRD,SynapseNotebook,NB_Infor_101_Ingest_Notebooks,4.940390e+06,105499.564,1,0,0,Billable
1,MDP-FABRIC-PRD,SynapseNotebook,NB_Infor_100_Refresh_ReplicationSet_And_GL,2.066267e+06,44836.920,1,0,0,Billable
2,MDP-FABRIC-PRD,SynapseNotebook,NB_LEM_402_Gold_Notebooks,1.468763e+06,55608.916,1,0,0,Billable
3,MDP-FABRIC-PRD,SynapseNotebook,NB_Delta_Table_Maintenance,1.261594e+06,48555.112,1,0,0,Billable
4,MDP-FABRIC-PRD,SynapseNotebook,NB_FieldForce_101_Ingest_Notebooks,8.909983e+05,27981.484,1,0,0,Billable


In [27]:
import pandas as pd

def clean_data(df):
    """Cleans and preprocesses the given DataFrame by:
    - Removing the last row if 'Item name' contains NaN
    - Dropping unnecessary columns
    - Renaming columns for consistency
    - Converting CU(s) to CU(h)

    Args:
        df (pd.DataFrame): The input DataFrame.

    Returns:
        pd.DataFrame: The cleaned DataFrame.
    """
    # Define columns to drop and rename
    columns_to_drop = {
        "Billing type",
        "Users",
        "Workspace",
        "Item kind",
        "Successful count",
        "Failed count",
        "Duration (s)",
    }

    rename_dict = {
        "Item name": "Item_name",
        "CU (s)": "CU(s)",
        "Duration (s)": "Duration(s)",
    }

    # Remove last row if 'Item name' is NaN
    if not df.empty and pd.isna(df.iloc[-1]["Item_name"]):
        df = df.iloc[:-1]

    # Drop unnecessary columns (ignore if they don't exist)
    df = df.drop(columns=columns_to_drop, errors="ignore")

    # Rename columns safely
    df = df.rename(columns=rename_dict)

    # Convert CU(s) to CU(h) if CU(s) exists
    if "CU(s)" in df.columns:
        df["CU(h)"] = df["CU(s)"] / 3600

    return df


In [28]:
df = clean_data(df)
df

,Item_name,CU(s),CU(h)
0,NB_Infor_101_Ingest_Notebooks,4.940390e+06,1372.330577
1,NB_Infor_100_Refresh_ReplicationSet_And_GL,2.066267e+06,573.962971
2,NB_LEM_402_Gold_Notebooks,1.468763e+06,407.989584
3,NB_Delta_Table_Maintenance,1.261594e+06,350.442647
4,NB_FieldForce_101_Ingest_Notebooks,8.909983e+05,247.499536
5,NB_Infor_100_GeneralLedgerTransactionDetail_Sm...,7.994571e+05,222.071407
6,NB_Infor_100_GeneralLedgerTransactionDetail_Sm...,5.611208e+05,155.866878
7,NB_LEM_101_Ingest_Notebooks,5.249229e+05,145.811905
8,NB_Infor_100_GeneralLedgerTransactionDetail_Sm...,4.209588e+05,116.932999
9,NB_Coupa_101_Ingest_Notebooks,3.867237e+05,107.423238


### CU Calculation basedon Sku Type

In [29]:
def calculate_CU_allocation(sku_capacity, period="day"):
    """Calculates CU allocation in seconds and hours for a given SKU capacity and time period.

    Args:
        sku_capacity (int): The CU capacity of the SKU (e.g., 64 for F64).
        period (str): The time period for calculation. Options: "hour", "day", "month", "year".

    Returns:
        dict: A dictionary with CU-seconds and CU-hours for the specified period.
    """
    # Time conversion factors
    seconds_per_minute = 60
    minutes_per_hour = 60
    hours_per_day = 24
    days_per_month = 30
    days_per_year = 365

    # Define period multipliers
    period_multipliers = {
        "hour": 1 / hours_per_day,  # 1/24 of a day
        "day": 1,
        "month": days_per_month,
        "year": days_per_year
    }

    if period not in period_multipliers:
        raise ValueError("Invalid period. Choose from 'hour', 'day', 'month', or 'year'.")

    # Compute total CU allocation
    multiplier = period_multipliers[period]
    total_CU_seconds = sku_capacity * seconds_per_minute * minutes_per_hour * hours_per_day * multiplier
    total_CU_hours = total_CU_seconds / 3600  # Convert to CU-hours

    return {
        "CU-seconds": total_CU_seconds,
        "CU-hours": total_CU_hours
    }

# Example usage
sku_capacity = 64  # Example SKU type (F64)
period = "day"  # Can be "hour", "day", "month", or "year"

cu_allocation = calculate_CU_allocation(sku_capacity, period)

print(
    f"Total CU Allocation for SKU {sku_capacity} ({period}):\n"
    f" - {cu_allocation['CU-seconds']:,} CU-seconds\n"
    f" - {cu_allocation['CU-hours']:,} CU-hours"
)

Total CU Allocation for SKU 64 (day):
 - 5,529,600 CU-seconds
 - 1,536.0 CU-hours


In [30]:
# Define global cost rates
TOTAL_DAILYCOST_PAYG = 11.52 * 24
TOTAL_DAILYCOST_RESERVED = 6.853 * 24
TOTAL_CU_HOURS = 1536
WEEK = 7
MONTH = 30
YEAR = 356


def calculate_execution_costs(df_filtered):
    """
    Calculate PAYG and Reserved cost per execution based on Compute Unit (CU) usage.

    Parameters:
    df_filtered (pd.DataFrame): Input DataFrame containing CU(h) column.
    total_CU_hours (int, optional): Total Compute Units per hour for the given capacity. Default is 1536 for F64.

    Returns:
    pd.DataFrame: Updated DataFrame with PAYG and Reserved costs.
    """

    # Calculate cost per execution
    df_filtered = df_filtered.copy()
    df_filtered["PAYG Cost($)"] = (
        (df_filtered["CU(h)"] / 14) / TOTAL_CU_HOURS
    ) * TOTAL_DAILYCOST_PAYG
    df_filtered["Reserved Cost($)"] = (
        (df_filtered["CU(h)"] / 14) / TOTAL_CU_HOURS
    ) * TOTAL_DAILYCOST_RESERVED

    print("Cost Per Day for Each Execution")
    return df_filtered


df_filtered = calculate_execution_costs(df)
df_filtered

Cost Per Day for Each Execution


,Item_name,CU(s),CU(h),PAYG Cost($),Reserved Cost($)
0,NB_Infor_101_Ingest_Notebooks,4.940390e+06,1372.330577,17.644250,10.496185
1,NB_Infor_100_Refresh_ReplicationSet_And_GL,2.066267e+06,573.962971,7.379524,4.389920
2,NB_LEM_402_Gold_Notebooks,1.468763e+06,407.989584,5.245580,3.120483
3,NB_Delta_Table_Maintenance,1.261594e+06,350.442647,4.505691,2.680339
4,NB_FieldForce_101_Ingest_Notebooks,8.909983e+05,247.499536,3.182137,1.892985
5,NB_Infor_100_GeneralLedgerTransactionDetail_Sm...,7.994571e+05,222.071407,2.855204,1.698499
6,NB_Infor_100_GeneralLedgerTransactionDetail_Sm...,5.611208e+05,155.866878,2.004003,1.192138
7,NB_LEM_101_Ingest_Notebooks,5.249229e+05,145.811905,1.874724,1.115233
8,NB_Infor_100_GeneralLedgerTransactionDetail_Sm...,4.209588e+05,116.932999,1.503424,0.894355
9,NB_Coupa_101_Ingest_Notebooks,3.867237e+05,107.423238,1.381156,0.821620
